In [31]:
from pysal.lib import weights
import geopandas as gpd
import pandas as pd
import WeightsUtilities as WU
import numpy as np
from scipy.spatial.distance import cdist
from arcpy.stats import GenerateSpatialWeightsMatrix
from arcpy.stats import SpatialAutocorrelation
import arcpy

In [28]:
shp_path="F:\大创数据\中间产出的数据\云南省和黄淮海平原已处理好的火点\黄淮海平原逐月火点\黄淮海平原已处理好的火点_1月.shp"
swm_path="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_1月.swm"
gdf = gpd.read_file(shp_path)

# 提取所需的字段（ID, X, Y, Z）
df = gdf[['ID', 'X', 'Y', 'Z']].copy()
coordinates = gdf[['X', 'Y', 'Z']].values

master_field="ID"
spatial_ref_name="World_Mercator"
# 设置距离阈值L0
L0 = 55000

# 计算空间权重矩阵
    #计算要素间的距离
    # coordinates = df[['X', 'Y']].values
distances = cdist(coordinates, coordinates)

    # 对得到的距离进行判断并赋值
spatial_weights = np.where((distances<=L0)&(distances!=0), 1, 0)

# 将空间权重矩阵保存到DataFrame中
spatial_weights_df = pd.DataFrame(spatial_weights)


# DataFrame转list转换函数
def df_to_list(df):
    result = []
    for idx, row in df.iterrows():
        element = idx
        neighbors = []
        weights = []
        for col, val in row.iteritems():
            if val != 0:
                neighbors.append(col)
                weights.append(val)
        result.append((element, neighbors, weights))
    return result
# DataFrame转dict转换函数
def df_to_dict(df):
    result = {}
    for idx, row in df.iterrows():
        element = idx
        neighbors = []
        weights = []
        for col, val in row.iteritems():
            if val != 0:
                neighbors.append(col)
                weights.append(val)
        result[element] = (np.array(neighbors), np.array(weights))
    return result

spatial_weights_dict=df_to_dict(spatial_weights_df)

#调用SWMWriter将spatial_weights_list写入swm文件
#格式如下：swm1.swm.writeEntry(要素,[邻居要素1,2,3……], [邻居要素的权重1,2,3……])
swm1=WU.SWMWriter(swm_path,master_field,spatial_ref_name,len(df),rowStandard=True)
for i in spatial_weights_dict:
    try: 
        if spatial_weights_dict[i][0] is None:
            swm1.swm.writeEntry(i,[],[])
        else:
            swm1.swm.writeEntry(i,spatial_weights_dict[i][0],spatial_weights_dict[i][1])
    except:
        print("---------",i,"---------")

swm1.close()
type(spatial_weights_dict[1][0])

numpy.ndarray

In [25]:
w={}
swm2=WU.SWMReader("D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_1月_geoscene.swm")
for i in range(swm2.numObs):
    info=swm2.swm.readEntry()
    w[info[0]]=[info[2],info[3]]
w

{29527: [array([29528, 20146, 20147, 46177, 62065, 67436, 29568, 29569, 29461,
         29813, 29812, 29816, 29460, 29825, 29824, 29506, 20118, 20117,
         29459, 29471]),
  array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
         0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])],
 29528: [array([29527, 20146, 20147, 46177, 62065, 29568, 29569, 67436, 29461,
         29813, 29816, 29812, 29460, 29825, 29824, 29506, 20118, 20117,
         29459]),
  array([0.05263158, 0.05263158, 0.05263158, 0.05263158, 0.05263158,
         0.05263158, 0.05263158, 0.05263158, 0.05263158, 0.05263158,
         0.05263158, 0.05263158, 0.05263158, 0.05263158, 0.05263158,
         0.05263158, 0.05263158, 0.05263158, 0.05263158])],
 29813: [array([29812, 46194, 46195,  4832, 29816, 20117, 29506, 20118, 29528,
         29527, 20146, 20147, 61586,    12]),
  array([0.07142857, 0.07142857, 0.07142857, 0.07142857, 0.07142857,
         0.07142857, 0.07142857, 0.07142857, 0.071428

In [32]:


arcpy.env.overwriteOutput = True
arcpy.env.workspace="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试"
featureset=arcpy.FeatureSet("F:\大创数据\中间产出的数据\云南省和黄淮海平原已处理好的火点\黄淮海平原逐月火点\黄淮海平原已处理好的火点_1月.shp")
SpatialAutocorrelation(featureset,Input_Field="FRP",Generate_Report=True,Conceptualization_of_Spatial_Relationships="GET_SPATIAL_WEIGHTS_FROM_FILE",
                       Weights_Matrix_File="D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\swm测试\黄淮海平原已处理好的火点_1月.swm")

id,value
0,0.062352
1,10.207621
2,0
3,C:\Users\Lenovo\AppData\Local\Temp\MoransI_Result_18896_16308_.html
